# All LSTM classification runs for Operator 1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score
from keras.models import Sequential # type: ignore
from keras.layers import Dense, LSTM, Dropout # type: ignore

In [2]:
rsrp_colors = ['#384959', '#6A89A7', '#88BDF2', '#BDDDFC']
rsrq_colors = ['#614419', '#B37E2E', '#DB9A39', '#FFB343']
sinr_colors = ['#135E4B', '#4CB572', '#A1D8B5', '#CCDCDB']
mode_colors = ['#872323', '#C93636', '#E26666', '#F4B6B6']

plt.rcParams.update({'font.size': 18})

## Processing functions

In [3]:
'''
    Reading the dataset and returning two pandas DataFrames:
        The first DataFrame containing every datapoint belonging to the given campaign (if any)
        The second DataFrame containing only one column - the mode (4G or 5G)
'''
def read_dataset(filename, campaigns=None, features=None):
    df = pd.read_csv(filename)
    df = df.loc[:, ~df.columns.str.match('Unnamed')]
    df = df.loc[:, ~df.columns.str.match('Timestamp')]
    df = df.replace('DC', 1)
    df = df.replace('LTE', 0)
    df = df[df['campaign'].str.contains('Driving') | df['campaign'].str.contains('Walking')]

    if campaigns != None:
        df = df[df['campaign'].isin(campaigns)]

    if features == None:
        features = ['RSRP', 'SINR', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ', 'campaign']

    features.append('Mode')
    features.append('Latitude')
    features.append('Longitude')
    return df[features]

'''
    Returns accuracy, recall and precision
'''
def metrics(y_true, y_pred):
    true_series, pred_series = [], []

    for i in range(len(y_pred)):
        seq_true, seq_pred = y_true[i], y_pred[i]
        for j in range(len(seq_pred)):
            true_series.append(seq_true[j])
            pred_series.append(seq_pred[j])

    return accuracy_score(true_series, pred_series), recall_score(true_series, pred_series), precision_score(true_series, pred_series)

## LSTM functions

In [4]:
'''
    Returns a numpy array of list 
'''
def series_split_sequences(f, t, n_steps_in, n_steps_out):
    X, y = [], []
    curr_campaign = ''
    for i in range(len(f)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out

        if out_end_ix > len(f):
            break

        if curr_campaign == '': 
            curr_campaign = f['campaign'].iloc[i]

        if f.iloc[i:out_end_ix]['campaign'].nunique() > 1:
            continue
    
        elif curr_campaign != f['campaign'].iloc[i]: 
            curr_campaign = f['campaign'].iloc[i]
        
        seq_x, seq_y = f[i:end_ix], t[end_ix:out_end_ix]
        
        X.append(seq_x.drop('campaign', axis=1))
        y.append(seq_y)

    return np.array(X), np.array(y)

def long_short_term_memory(train, test, target_feature:str, features:list, n_steps_in=5, n_steps_out=1, units=[5, 10]):

    features.remove('Mode')
    
    X, y = series_split_sequences(train[features], train[target_feature], n_steps_in=n_steps_in, n_steps_out=n_steps_out)
    X_test, y_test = series_split_sequences(test[features], test[target_feature], n_steps_in=n_steps_in, n_steps_out=n_steps_out)

    features.remove('campaign')

    train_idx = 4 * len(y) // 5

    X_train, y_train = X[0 : train_idx], y[0 : train_idx]
    X_val, y_val = X[train_idx ::], y[train_idx ::]

    model = Sequential()
    if len(units) > 1:
        for unit in units[0:-1]:
            model.add(LSTM(units=unit,
                        activation='relu', recurrent_activation='sigmoid',
                        return_sequences=True, return_state=False
                        ))
            model.add(Dropout(0.2))
    model.add(LSTM(units=units[-1],
                   activation='relu', recurrent_activation='sigmoid',
                   return_sequences=False, return_state=False
                   ))
    model.add(Dropout(0.2))
    model.add(Dense(n_steps_out, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), shuffle=True, verbose=0)
    
    y_pred = model.predict(X_test, verbose=0)
    y_pred = (y_pred >= 0.5).astype(int)

    train_pred = model.predict(X, verbose=0)
    train_pred = (train_pred >= 0.5).astype(int)

    if n_steps_out > 1:
        y_pred_plot, y_true_plot, train_pred_plot, train_true_plot = y_pred, y_test, train_pred, y_train
    else:
        # test data
        y_true_plot = test[target_feature].values
        y_pred_plot = np.empty_like(test)
        y_pred_plot[:, :] = np.nan
        y_pred_plot[n_steps_in : len(y_pred) + n_steps_in, :] = y_pred

        # train data
        train_true_plot = train[target_feature].values
        train_pred_plot = np.empty_like(train)
        train_pred_plot[:, :] = np.nan
        train_pred_plot[n_steps_in : len(train_pred) + n_steps_in, :] = train_pred
    
    accuracy, recall, precision = metrics(y_pred=y_pred, y_true=y_test)
    train_accuracy, train_recall, train_precision = metrics(y_pred=train_pred, y_true=y)

    acc = []
    train_acc = []

    for i in range(n_steps_out):
        acc.append(accuracy_score(y_test[:, i], y_pred[:, i]))
        train_acc.append(accuracy_score(y[:, i], train_pred[:, i]))
    
    results = {'y_pred': y_pred,
               'y_true': y_test,
               'train_true': y,
               'train_pred': train_pred,
               'y_pred_plot': y_pred_plot,
               'y_true_plot': y_true_plot,
               'train_pred_plot': train_pred_plot,
               'train_true_plot': train_true_plot,
               'Accuracy': round(accuracy, 3),
               'Precision': round(precision, 3),
               'Recall': round(recall, 3),
               'Accuracy_train': round(train_accuracy, 3),
               'Precision_train': round(train_precision, 3),
               'Recall_train': round(train_recall, 3),
               'Accuracy_list': acc,
               'Accuracy_train_list': train_acc
               }

    return results

def plot_mode(pred, true, operator:str, comb:str, n_steps_in:int, n_steps_out:int, test_train='Test'):
    fig = plt.figure(figsize=(16, 6))
    pred = plt.plot(np.array(pred), color=mode_colors[1], label='Predicted', zorder=2)
    true = plt.plot(np.array(true) + 2, color=mode_colors[3], label='True', zorder=3)
    plt.xlabel('Index / Timestamp')
    plt.ylabel('HO')
    plt.ylim((-0.2, 3.6))
    plt.yticks([0, 1, 2, 3], ['4G', '5G', '4G', '5G'])
    plt.title(f'{test_train} data -- {operator}\nTraining features: {comb}\nLook-back: {n_steps_in}, Looh-ahead: {n_steps_out}')
    plt.legend(handles=[pred[0], true[0]], ncols=2, loc='best')
    plt.show()
    return fig

'''
    Plots the accuracy in a histogram for test and training data
'''
def plot_accuracy_timesteps(true, pred, train_true, train_pred, comb:list, n_steps_out:int, operator:str):
    acc = []
    train_acc = []

    for i in range(n_steps_out):
        acc.append(accuracy_score(true[:, i], pred[:, i]))
        train_acc.append(accuracy_score(train_true[:, i], train_pred[:, i]))

    print(acc)
    print(train_acc)

    labels = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th', '13th', '14th', '15th', '16th', '17th', '18th', '19th', '20th']
    x = np.arange(n_steps_out)

    width = 0.35

    fig, ax = plt.subplots(figsize=(16, 6))

    bars1 = ax.bar(x - width/2, acc, width, label='Accuracy', color=mode_colors[2], edgecolor='white')

    bars2 = ax.bar(x + width/2, train_acc, width, label='Training Accuracy', color=mode_colors[3], edgecolor='white')

    ax.set_ylabel('Accuracy')
    ax.set_xlabel('Step ahead')
    ax.set_title(f'{operator}\nAccuracy for each timestep ahead\nCombination of training features: {comb}')
    ax.set_xticks(x)
    ax.set_ylim(0, 1)
    ax.set_xticklabels(labels[0:n_steps_out])
    ax.legend(loc='lower right')

    def add_value_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.3f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom')

    add_value_labels(bars1)
    add_value_labels(bars2)

    plt.show()
    return fig


'''
    Plots the accuracy for each timestep ahead for both test and training data
'''
def boxplot_accuracies(results:dict, n_steps_out, whiskers=[5, 95]):
    accuracy = [[] for _ in range(n_steps_out)]
    accuracy_train = [[] for _ in range(n_steps_out)]

    for key in results:
        for i in range(n_steps_out):
            accuracy[i].append(results[key]['Accuracy_list'][i])
            accuracy_train[i].append(results[key]['Accuracy_train_list'][i])

    labels = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th', '13th', '14th', '15th', '16th', '17th', '18th', '19th', '20th']

    fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
    sns.boxplot(data=np.array(accuracy).T, color=mode_colors[2], ax=axes[0], whis=whiskers, zorder=2)
    axes[0].set_title('Accuracy for test data')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_xlabel('Steps ahead')
    axes[0].set_xticklabels(labels[0:n_steps_out])
    axes[0].grid(zorder=0)

    sns.boxplot(data=np.array(accuracy_train).T, color=mode_colors[3], ax=axes[1], whis=whiskers, zorder=3)
    axes[1].set_title('Accuracy for train data')
    axes[1].set_ylabel('Training Accuracy')
    axes[1].set_xlabel('Steps ahead')
    axes[1].set_xticklabels(labels[0:n_steps_out])
    axes[1].grid(zorder=0)

    plt.show()
    return fig

In [5]:
# Parameters
tf.random.set_seed(4567)
dataset = 'datasets/Op2_merged.csv'
operator = 'Operator 2'

In [6]:
overlap = ['Ping_UNIDATA_4G5G_Vodafone_Driving_Viale_Marconi_Test_2_No_Flight_Mode', 
           'Ping_8888_4G5G_Vodafone_Driving_Viale_Marconi_No_Flight_Mode', 
           'Capacity_Ookla_UNIDATA_4G5G_Vodafone_Driving_Viale_Marconi', 
           'Capacity_Ookla_4G5G_Vodafone_Driving_Appia_End_to_Eur_Palasport_No_Flight_Mode']

test = read_dataset(dataset, campaigns=[overlap[0]])
train = read_dataset(dataset, campaigns=overlap[1:3])
test_campaigns = test['campaign'].unique().tolist()

# only a part of this route overlaps with the others
route_to_add = read_dataset(dataset, campaigns=[overlap[3]])
test = pd.concat([route_to_add[5350::], test])
test.reset_index(drop=True, inplace=True)

campaigns = train['campaign'].unique().tolist()
test_campaigns = test['campaign'].unique().tolist()

In [7]:
target_features = ['Mode']
units = [10, 10, 5]
feature_selection = [['RSRP'], ['SINR'], ['RSRQ'], 
                     ['RSRP', 'SINR'], ['RSRP', 'RSRQ'], ['SINR', 'RSRQ'], 
                     ['RSRP', 'SINR', 'RSRQ'],
                     ['SSS_RSRP'], ['SSS_SINR'], ['SSS_RSRQ'],
                     ['SSS_RSRP', 'SSS_SINR'], ['SSS_RSRP', 'SSS_RSRQ'], ['SSS_SINR', 'SSS_RSRQ'], 
                     ['SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ'],
                     ['RSRP', 'SSS_RSRP'], ['SINR', 'SSS_SINR'], ['RSRQ', 'SSS_RSRQ'], 
                     ['RSRP', 'SINR', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

In [8]:
n_steps_ins, n_steps_out = [1, 2, 5], 1

step1_results = {}

for n_steps_in in n_steps_ins:
    temp_results = {}
    for target in target_features:
        for features in feature_selection:
            used_features = f''
            if len(features) == 6:
                used_features = 'All'
            else:
                used_features = ', '.join(features)
            features.append('campaign')
            features.append('Mode')
            warnings.simplefilter(action='ignore')
            temp_results[used_features] = long_short_term_memory(train=train[features], test=test[features], target_feature=target, features=features, n_steps_in=n_steps_in, n_steps_out=n_steps_out, units=units)
    step1_results[f'x:{n_steps_in}'] = temp_results

In [9]:

x1n1 = step1_results['x:1']

x1n1_results = pd.DataFrame.from_dict(x1n1, orient='index')
x1n1_results.reset_index(inplace=True)
x1n1_results.rename(columns={'index': 'Combination'}, inplace=True)
x1n1_results = x1n1_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x1n1_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [10]:
x2n1 = step1_results['x:2']

x2n1_results = pd.DataFrame.from_dict(x1n1, orient='index')
x2n1_results.reset_index(inplace=True)
x2n1_results.rename(columns={'index': 'Combination'}, inplace=True)
x2n1_results = x2n1_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x2n1_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [11]:
x3n1 = step1_results['x:5']

x3n1_results = pd.DataFrame.from_dict(x1n1, orient='index')
x3n1_results.reset_index(inplace=True)
x3n1_results.rename(columns={'index': 'Combination'}, inplace=True)
x3n1_results = x3n1_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x3n1_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [12]:
n_steps_ins, n_steps_out = [1, 2, 5], 2

step2_results = {}

for n_steps_in in n_steps_ins:
    temp_results = {}
    for target in target_features:
        for features in feature_selection:
            used_features = f''
            if len(features) == 6:
                used_features = 'All'
            else:
                used_features = ', '.join(features)
            features.append('campaign')
            features.append('Mode')
            warnings.simplefilter(action='ignore')
            temp_results[used_features] = long_short_term_memory(train=train[features], test=test[features], target_feature=target, features=features, n_steps_in=n_steps_in, n_steps_out=n_steps_out, units=units)
    step2_results[f'x:{n_steps_in}'] = temp_results

In [13]:
x1n2 = step2_results['x:1']

x1n2_results = pd.DataFrame.from_dict(x1n1, orient='index')
x1n2_results.reset_index(inplace=True)
x1n2_results.rename(columns={'index': 'Combination'}, inplace=True)
x1n2_results = x1n2_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x1n2_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [14]:
x2n2 = step2_results['x:2']

x2n2_results = pd.DataFrame.from_dict(x1n1, orient='index')
x2n2_results.reset_index(inplace=True)
x2n2_results.rename(columns={'index': 'Combination'}, inplace=True)
x2n2_results = x2n2_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x2n2_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [15]:
x5n2 = step2_results['x:5']

x5n2_results = pd.DataFrame.from_dict(x1n1, orient='index')
x5n2_results.reset_index(inplace=True)
x5n2_results.rename(columns={'index': 'Combination'}, inplace=True)
x5n2_results = x5n2_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x5n2_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [16]:
n_steps_ins, n_steps_out = [1, 3, 5], 3

step3_results = {}

for n_steps_in in n_steps_ins:
    temp_results = {}
    for target in target_features:
        for features in feature_selection:
            used_features = f''
            if len(features) == 6:
                used_features = 'All'
            else:
                used_features = ', '.join(features)
            features.append('campaign')
            features.append('Mode')
            warnings.simplefilter(action='ignore')
            temp_results[used_features] = long_short_term_memory(train=train[features], test=test[features], target_feature=target, features=features, n_steps_in=n_steps_in, n_steps_out=n_steps_out, units=units)
    step3_results[f'x:{n_steps_in}'] = temp_results

In [17]:
x1n3 = step3_results['x:1']

x1n3_results = pd.DataFrame.from_dict(x1n1, orient='index')
x1n3_results.reset_index(inplace=True)
x1n3_results.rename(columns={'index': 'Combination'}, inplace=True)
x1n3_results = x1n3_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x1n3_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [18]:
x3n3 = step3_results['x:3']

x3n3_results = pd.DataFrame.from_dict(x1n1, orient='index')
x3n3_results.reset_index(inplace=True)
x3n3_results.rename(columns={'index': 'Combination'}, inplace=True)
x3n3_results = x3n3_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x3n3_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [19]:
x5n3 = step3_results['x:5']

x5n3_results = pd.DataFrame.from_dict(x1n1, orient='index')
x5n3_results.reset_index(inplace=True)
x5n3_results.rename(columns={'index': 'Combination'}, inplace=True)
x5n3_results = x5n3_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x5n3_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [20]:
n_steps_ins, n_steps_out = [2, 5, 8], 5

step5_results = {}

for n_steps_in in n_steps_ins:
    temp_results = {}
    for target in target_features:
        for features in feature_selection:
            used_features = f''
            if len(features) == 6:
                used_features = 'All'
            else:
                used_features = ', '.join(features)
            features.append('campaign')
            features.append('Mode')
            warnings.simplefilter(action='ignore')
            temp_results[used_features] = long_short_term_memory(train=train[features], test=test[features], target_feature=target, features=features, n_steps_in=n_steps_in, n_steps_out=n_steps_out, units=units)
    step5_results[f'x:{n_steps_in}'] = temp_results

In [21]:
x2n5 = step5_results['x:2']

x2n5_results = pd.DataFrame.from_dict(x1n1, orient='index')
x2n5_results.reset_index(inplace=True)
x2n5_results.rename(columns={'index': 'Combination'}, inplace=True)
x2n5_results = x2n5_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x2n5_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [22]:
x5n5 = step5_results['x:5']

x5n5_results = pd.DataFrame.from_dict(x1n1, orient='index')
x5n5_results.reset_index(inplace=True)
x5n5_results.rename(columns={'index': 'Combination'}, inplace=True)
x5n5_results = x5n5_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x5n5_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [23]:
x8n5 = step5_results['x:8']

x8n5_results = pd.DataFrame.from_dict(x1n1, orient='index')
x8n5_results.reset_index(inplace=True)
x8n5_results.rename(columns={'index': 'Combination'}, inplace=True)
x8n5_results = x8n5_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x8n5_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [24]:
n_steps_ins, n_steps_out = [2, 5, 10, 15], 10

step10_results = {}

for n_steps_in in n_steps_ins:
    temp_results = {}
    for target in target_features:
        for features in feature_selection:
            used_features = f''
            if len(features) == 6:
                used_features = 'All'
            else:
                used_features = ', '.join(features)
            features.append('campaign')
            features.append('Mode')
            warnings.simplefilter(action='ignore')
            temp_results[used_features] = long_short_term_memory(train=train[features], test=test[features], target_feature=target, features=features, n_steps_in=n_steps_in, n_steps_out=n_steps_out, units=units)
    step10_results[f'x:{n_steps_in}'] = temp_results

In [25]:
x2n10 = step10_results['x:2']

x2n10_results = pd.DataFrame.from_dict(x1n1, orient='index')
x2n10_results.reset_index(inplace=True)
x2n10_results.rename(columns={'index': 'Combination'}, inplace=True)
x2n10_results = x2n10_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x2n10_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [26]:
x5n10 = step10_results['x:5']

x5n10_results = pd.DataFrame.from_dict(x1n1, orient='index')
x5n10_results.reset_index(inplace=True)
x5n10_results.rename(columns={'index': 'Combination'}, inplace=True)
x5n10_results = x5n10_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x5n10_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [27]:
x10n10 = step10_results['x:10']

x10n10_results = pd.DataFrame.from_dict(x1n1, orient='index')
x10n10_results.reset_index(inplace=True)
x10n10_results.rename(columns={'index': 'Combination'}, inplace=True)
x10n10_results = x10n10_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x10n10_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496


In [28]:
x15n10 = step10_results['x:15']

x15n10_results = pd.DataFrame.from_dict(x1n1, orient='index')
x15n10_results.reset_index(inplace=True)
x15n10_results.rename(columns={'index': 'Combination'}, inplace=True)
x15n10_results = x15n10_results.drop(['y_pred', 'y_true', 'train_true', 'train_pred', 'y_pred_plot', 'y_true_plot', 'train_pred_plot', 'train_true_plot', 'Accuracy_list', 'Accuracy_train_list'], axis=1)
x15n10_results

,Combination,Accuracy,Precision,Recall,Accuracy_train,Precision_train,Recall_train
0,RSRP,0.406,1.000,0.012,0.517,1.000,0.017
1,SINR,0.505,0.586,0.599,0.611,0.570,0.847
2,RSRQ,0.540,0.589,0.777,0.503,0.497,0.864
3,"RSRP, SINR",0.444,0.582,0.264,0.587,0.651,0.342
4,"RSRP, RSRQ",0.402,1.000,0.005,0.517,1.000,0.017
5,"SINR, RSRQ",0.506,0.598,0.541,0.629,0.590,0.801
6,"RSRP, SINR, RSRQ",0.491,0.606,0.436,0.647,0.688,0.517
7,SSS_RSRP,0.433,0.994,0.056,0.528,0.991,0.039
8,SSS_SINR,0.646,0.815,0.532,0.793,0.833,0.725
9,SSS_RSRQ,0.572,0.866,0.340,0.725,0.900,0.496
